In [2]:
from zipfile import ZipFile
import numpy as np
import pandas as pd
import zipfile
import re
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib
from sklearn.neighbors import KernelDensity
import os


In [3]:
zf = zipfile.ZipFile('Data/Data_incidents.csv.zip') 
data = pd.read_csv('Data/Data_incidents.csv.zip').reset_index()

/Users/Anna/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Plot development of all incidents and mass shooting over time (year-month)

### Mass shootings

In [4]:
data = data.drop(columns=['index','Unnamed: 0'])

In [5]:
mass_shootings = data[data['Mass Shooting']==True]

In [6]:
ms_time = mass_shootings.groupby('Date')['incident_id'].nunique()

In [7]:
ms_time = ms_time.to_frame()
ms_time.reset_index(level=0, inplace=True)

In [8]:
#check sum
ms_time['incident_id'].sum()

1352

In [9]:
# format date
ms_time['date'] = pd.to_datetime(ms_time.Date.str.strip(), format = '%Y-%m-%d')#
ms_time=ms_time.drop(columns = 'Date');

In [10]:
# Create new date columns
ms_time['Year'] = ms_time.date.apply(lambda x: x.year)
ms_time['Month'] = ms_time.date.apply(lambda x: x.month)
ms_time['Year_Month'] = ms_time.date.apply(lambda x: 100*x.year + x.month)

In [11]:
ms_year_month = ms_time.groupby('Year_Month').incident_id.sum()

In [12]:
ms_year_month = ms_year_month.to_frame()
ms_year_month.reset_index(level=0, inplace=True)
ms_year_month.columns = ['Year_Month', 'ms_all']

### All incidents

In [13]:
inc_all = data.Date
inc_all = inc_all.to_frame()
inc_all['counts'] = 1

In [14]:
inc_all['date'] = pd.to_datetime(inc_all.Date.str.strip(), format = '%Y-%m-%d')#
inc_all= inc_all.drop(['Date'], axis=1);

In [15]:
inc_all['Year'] = inc_all.date.apply(lambda x: x.year)
inc_all['Year_Month'] = inc_all.date.apply(lambda x: 100*x.year + x.month)

In [16]:
all_YM = inc_all.groupby('Year_Month').counts.sum()

In [17]:
all_YM_pd = all_YM.to_frame()
all_YM_pd.reset_index(level=0, inplace=True)
all_YM_pd.columns = ['Year_Month', 'inc_all']

In [18]:
YM = pd.merge(all_YM_pd, ms_year_month, on='Year_Month')

In [19]:
YM.to_csv('Data/YM.csv')
# The plot is made in Excel

## TABLE OF CHARACTERISTICS

In [20]:
dataframes_gender = []

for row in mass_shootings['participant_gender'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T
    #print('participants: ', participants)

    data = dict(zip(*participants))
    #print('data: ', data)

    dataframe = pd.Series(data).to_frame().T
    #display(dataframe)

    dataframes_gender.append(dataframe)

gender = pd.concat(dataframes_gender).reset_index().drop('index', 1).fillna('')

/Users/Anna/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [21]:
dataframes_age = []

for row in mass_shootings['participant_age'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T
    #print('participants: ', participants)

    data = dict(zip(*participants))
    #print('data: ', data)

    dataframe = pd.Series(data).to_frame().T
    #display(dataframe)

    dataframes_age.append(dataframe)

age = pd.concat(dataframes_age).reset_index().drop('index', 1).fillna('')

/Users/Anna/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [22]:
dataframes_status = []

for row in mass_shootings['participant_status'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T

    data = dict(zip(*participants))

    dataframe = pd.Series(data).to_frame().T

    dataframes_status.append(dataframe)

status = pd.concat(dataframes_status).reset_index().drop('index', 1).fillna('')

/Users/Anna/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [23]:
dataframes_type = []

for row in mass_shootings['participant_type'].fillna("0:,")[:len(mass_shootings)]:
    participants = np.array([re.split(r'::?', p) for p in re.split(r'\|\|?', row)]).T
    #print('participants: ', participants)

    data = dict(zip(*participants))
    #print('data: ', data)

    dataframe = pd.Series(data).to_frame().T
    #display(dataframe)

    dataframes_type.append(dataframe)

_type = pd.concat(dataframes_type).reset_index().drop('index', 1).fillna('')

/Users/Anna/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


### Gender of victimes and suspects

In [24]:
combined_gender = gender.astype(str).add(',').add(_type.astype(str))
com = combined_gender.fillna(',')

com['Male, Victim'] = com.apply(lambda row: sum(row[0:-1]=='Male,Victim') ,axis=1)
com['Female, Victim'] = com.apply(lambda row: sum(row[0:-1]=='Female,Victim') ,axis=1)
com['Male, Suspect'] = com.apply(lambda row: sum(row[0:-1]=='Male,Subject-Suspect') ,axis=1)
com['Female, Suspect'] = com.apply(lambda row: sum(row[0:-1]=='Female,Subject-Suspect') ,axis=1)
com['Victim1'] = com.apply(lambda row: sum(row[0:-1]==',Victim') ,axis=1)
com['Victim2'] = com.apply(lambda row: sum(row[0:-1]==',,Victim') ,axis=1)
com['Subject'] = com.apply(lambda row: sum(row[0:-1]==',Subject-Suspect') ,axis=1)

com[['Male, Victim','Female, Victim','Male, Suspect','Female, Suspect','Victim1','Victim2','Subject']].sum()

Male, Victim       3834
Female, Victim     1479
Male, Suspect      1073
Female, Suspect      32
Victim1            1367
Victim2             108
Subject             105
dtype: int64

### Status of victimes and suspects

In [25]:
combined_status = status.astype(str).add(', ').add(_type.astype(str))
com_s = combined_status.fillna(',')

com_s['Killed, Victim'] = com_s.apply(lambda row: sum(row[0:-1]=='Killed, Victim') ,axis=1)
com_s['Injured, Victim'] = com_s.apply(lambda row: sum(row[0:-1]=='Injured, Victim') ,axis=1)
com_s['Unharmed/Arrested, Victim'] = com_s.apply(lambda row: sum(row[0:-1]=='Unharmed, Arrested, Victim') ,axis=1)
com_s['Killed, Suspect'] = com_s.apply(lambda row: sum(row[0:-1]=='Killed, Subject-Suspect') ,axis=1)
com_s['Injured, Suspect'] = com_s.apply(lambda row: sum(row[0:-1]=='Injured, Subject-Suspect') ,axis=1)
com_s['Unharmed/Arrested, Suspect'] = com_s.apply(lambda row: sum(row[0:-1]=='Unharmed, Arrested, Subject-Suspect') ,axis=1)

com_s[['Killed, Victim', 'Injured, Victim','Unharmed/Arrested, Victim','Killed, Suspect','Injured, Suspect','Unharmed/Arrested, Suspect']].sum()

Killed, Victim                1426
Injured, Victim               5328
Unharmed/Arrested, Victim        3
Killed, Suspect                 92
Injured, Suspect                25
Unharmed/Arrested, Suspect     550
dtype: int64

### Combined gender and status

In [26]:
combined_gender_status = gender.astype(str).add(',').add(status.astype(str)) #.add(', ').add(status.astype(str))
com_gs = combined_gender_status.fillna(',')

com_gs['Male, Killed'] = com_gs.apply(lambda row: sum(row[0:103]=='Male,Killed') ,axis=1)
com_gs['Female, Killed'] = com_gs.apply(lambda row: sum(row[0:103]=='Female,Killed') ,axis=1)
com_gs['Male, Injured'] = com_gs.apply(lambda row: sum(row[0:103]=='Male,Injured') ,axis=1)
com_gs['Female, Injured'] = com_gs.apply(lambda row: sum(row[0:103]=='Female,Injured') ,axis=1)
#com_gs['Victim1'] = com_gs.apply(lambda row: sum(row[0:103]==',Victim') ,axis=1)
#com_gs['Victim2'] = com_gs.apply(lambda row: sum(row[0:103]==',,Victim') ,axis=1)
#com_gs['Subject'] = com_gs.apply(lambda row: sum(row[0:103]==',Subject-Suspect') ,axis=1)

com_gs[['Male, Killed','Female, Killed','Male, Injured','Female, Injured']].sum()

Male, Killed       1074
Female, Killed      417
Male, Injured      2837
Female, Injured    1057
dtype: int64

# Most frequent characteristics

In [109]:
df_incidents = mass_shootings.iloc[:, list(range(18,127))]
incidents = []

for col in df_incidents.columns: 
    incidents.append(col)

In [111]:
char = pd.DataFrame(mass_shootings[incidents].sum())
char.columns = ['count']
char = char[char['count'] >= 50]

In [112]:
char.sort_values('count', ascending=False)

,count
Mass Shooting,1352.0
Shot - Wounded/Injured,1290.0
Shot - Dead,708.0
Bar/club incident - in or around establishment,228.0
Drive-by,213.0
Gang involvement,157.0
Possession,137.0
Domestic Violence,93.0
Institution/Group/Business,87.0
Mass Murder,80.0
